In [2]:
import deepchem as dc
from deepchem.feat.graph_data import GraphData
from deepchem.feat import MolGraphConvFeaturizer
import numpy as np
import pandas as pd

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (C:\Users\yyyyx\miniconda3\envs\deepchem\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [31]:
df = pd.read_csv('./HIV_protease_hERG.csv')

In [36]:
df

,Name,Structure,hERG inhibitor,hERG inhibitor_binary
0,CHEMBL729/LOPINAVIR,CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...,Yes,1.0
1,CHEMBL584/NELFINAVIR,CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...,No,0.0
2,CHEMBL57/NEVIRAPINE,CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1,No,0.0
3,CHEMBL282042/SAQUINAVIR MESYLATE,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,NaN,NaN
4,CHEMBL222559/TIPRANAVIR,CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...,No,0.0
...,...,...,...,...
108,CHEMBL100089,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,No,0.0
109,CHEMBL100048,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,No,0.0
110,CHEMBL100040,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,No,0.0
111,CHEMBL100039,CC(C)CCN(C[C@@H](O)[C@H](CC1=CC=CC=C1)NC(=O)[C...,No,0.0


In [10]:
model = dc.models.GCNModel(n_tasks=1, 
    graph_conv_layers=[128, 128],
    dense_layer_size=256,
    dropout=0.2,
    mode='classification',
    learning_rate=0.001,
    batch_size=128)

In [11]:
model.restore('./hERG_Block/model_3rd.pt/checkpoint1.pt')

In [12]:
model

In [16]:
# 1. SMILES에서 그래프 형식으로 변환
featurizer = MolGraphConvFeaturizer()
features = featurizer.featurize(df["Structure"])  # Structure 컬럼에서 특징 추출

In [17]:
# 2. 레이블 설정 (pIC50)
labels = None

In [18]:
# 3. 가중치 (필요시 기본값으로 1 설정)
weights = None  # 기본적으로 None으로 설정. 커스텀 가중치가 있으면 지정.

# 4. 데이터셋 생성
dataset = dc.data.NumpyDataset(X=features, y=labels, w=weights)

In [19]:
# 데이터 크기 확인
print(f"Feature shape: {features[0].node_features.shape}")


Feature shape: (46, 30)


In [23]:
# 예측 수행
y_pred_prob = model.predict(dataset)

In [24]:
# 가장 큰 값을 가진 인덱스를 클래스로 결정
y_pred = np.argmax(y_pred_prob, axis=1)

In [26]:
y_pred_prob

array([[0.4515799 , 0.5484201 ],
       [0.15127683, 0.8487231 ],
       [0.91629505, 0.08370488],
       [0.61045724, 0.38954273],
       [0.43620017, 0.5637998 ],
       [0.81627107, 0.18372896],
       [0.954685  , 0.04531498],
       [0.9632999 , 0.03670006],
       [0.9907074 , 0.00929259],
       [0.8684004 , 0.13159956],
       [0.9739327 , 0.02606724],
       [0.34888387, 0.65111613],
       [0.62212014, 0.37787983],
       [0.9457412 , 0.05425883],
       [0.5175923 , 0.48240766],
       [0.92422837, 0.07577164],
       [0.9879326 , 0.01206732],
       [0.9510661 , 0.04893399],
       [0.66605425, 0.33394578],
       [0.9879326 , 0.01206732],
       [0.9925171 , 0.00748285],
       [0.7864645 , 0.21353556],
       [0.63766384, 0.36233613],
       [0.85627043, 0.14372957],
       [0.7065993 , 0.29340073],
       [0.9453644 , 0.05463562],
       [0.20567337, 0.79432666],
       [0.92250484, 0.07749511],
       [0.9323995 , 0.06760045],
       [0.84406555, 0.15593444],
       [0.

In [37]:
# 예측 수행
df['Prediction'] = y_pred

In [38]:
df

,Name,Structure,hERG inhibitor,hERG inhibitor_binary,Prediction
0,CHEMBL729/LOPINAVIR,CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...,Yes,1.0,1
1,CHEMBL584/NELFINAVIR,CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...,No,0.0,1
2,CHEMBL57/NEVIRAPINE,CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1,No,0.0,0
3,CHEMBL282042/SAQUINAVIR MESYLATE,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,NaN,NaN,0
4,CHEMBL222559/TIPRANAVIR,CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...,No,0.0,1
...,...,...,...,...,...
108,CHEMBL100089,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,No,0.0,1
109,CHEMBL100048,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,No,0.0,0
110,CHEMBL100040,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,No,0.0,0
111,CHEMBL100039,CC(C)CCN(C[C@@H](O)[C@H](CC1=CC=CC=C1)NC(=O)[C...,No,0.0,0


In [39]:
df['Prediction'].sum()

11

In [40]:
df.to_csv('./HIV114_hERG_prediction.csv')

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# StandardScaler를 사용해 Prediction 값을 원복
# mean = scaler.mean_[0]  # StandardScaler에서 사용된 평균
# std = scaler.scale_[0]  # StandardScaler에서 사용된 표준편차

# 원복 공식 적용
df["Original Prediction"] = df["Prediction"] * 2.0954617919366605 -3.0520097604259093

# 결과 확인
print(df.head())


                               Name  \
0               CHEMBL729/LOPINAVIR   
1              CHEMBL584/NELFINAVIR   
2               CHEMBL57/NEVIRAPINE   
3  CHEMBL282042/SAQUINAVIR MESYLATE   
4           CHEMBL222559/TIPRANAVIR   

                                           Structure Solubility  Prediction  \
0  CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...    Soluble   -4.426832   
1  CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...   Moderate   -4.294361   
2              CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1   Moderate   -3.793959   
3  CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...        NaN   -2.228862   
4  CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...  Insoluble   -4.859251   

   Original Prediction  
0           -12.328266  
1           -12.050678  
2           -11.002106  
3            -7.722506  
4           -13.234385  


In [23]:
mean = scaler.mean_[0]
std = scaler.scale_[0]

AttributeError: 'StandardScaler' object has no attribute 'mean_'

In [25]:
df

,Name,Structure,Solubility,Prediction,Original Prediction
0,CHEMBL729/LOPINAVIR,CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...,Soluble,-5.241626,-14.035636
1,CHEMBL584/NELFINAVIR,CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...,Moderate,-5.525194,-14.629843
2,CHEMBL57/NEVIRAPINE,CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1,Moderate,-3.309443,-9.986820
3,CHEMBL282042/SAQUINAVIR MESYLATE,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,NaN,-5.772190,-15.147413
4,CHEMBL222559/TIPRANAVIR,CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...,Insoluble,-7.282715,-18.312662
...,...,...,...,...,...
108,CHEMBL100089,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,Moderate,-6.030370,-15.688419
109,CHEMBL100048,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Soluble,-3.489471,-10.364064
110,CHEMBL100040,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Moderate,-5.755380,-15.112189
111,CHEMBL100039,CC(C)CCN(C[C@@H](O)[C@H](CC1=CC=CC=C1)NC(=O)[C...,Soluble,-4.615815,-12.724273
